# 作业三：图像分类
使用food-11食品分类数据集进行分类，模型不限，使用了卷积神经网络即可
目的是刷到尽可能高的准确率，可以使用数据增强、交叉验证、集成学习等一系列trick

In [1]:
# 数据集
! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
! unzip -q food11.zip

--2023-01-06 20:51:59--  https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/6l2vcvxl54b0b6w/food11.zip [following]
--2023-01-06 20:52:00--  https://www.dropbox.com/s/raw/6l2vcvxl54b0b6w/food11.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab5f19579307e6a523107d4729.dl.dropboxusercontent.com/cd/0/inline/B0ADBbsGKwAZW2Al89fJoivWxJl7IQV2PClKkLloKzSwwpgv6svQ-POkqNtxmGqz2t844ozFzo7fdgH6EAR6BcJt5Nr1VWFWBa3P9kYea43ckOr8GYCu8jLDTogyHakyqpUhIzhB32aLXSVfsE5yjQK3fxiCoklTaDDIu_NFaSs2-Q/file# [following]
--2023-01-06 20:52:00--  https://ucab5f19579307e6a523107d4729.dl.dropboxusercontent.com/cd/0/inline/B0ADBbsGKwAZW2Al89fJoivWxJl7IQV2PClKkLloKzSwwpgv6svQ-POkqNtxmGqz2t844ozFzo7fdgH6EAR

### 导包与函数定义

In [17]:
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

import time
from tqdm import tqdm
import random

_exp_name = "resnet50_first_simple_try"
myseed = 666              # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

### 定义对图片的变换操作（数据增强）

In [12]:
# 对验证集和测试集一般不做数据增强，只需对其做resize并转换为Tensor即可
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

# 但在测试阶段也是可以做数据增强的。可以使用下面的trian_tfm转换方法在测试阶段制造一系列的图片，然后做测试阶段数据增强。如：Final Prediction = avg_train_tfm_pred * 0.5 + test_tfm_pred* 0.5
train_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    # 相较baseline代码，这里添加了随机平行翻转和随机旋转
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    # 相较baseline代码，这里添加了灰度值归一化
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

### 数据集定义与加载
由于该数据集图片的名字就包含了类别（0_4.jpg代表第0类第4张），所以利用`__getitem__`来获取label

In [4]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files=None) -> None:
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        fname = self.files[index]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

### 定义模型

In [13]:
import torchvision

# load resnet model with torhvision lib
# Note: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.

# net = torchvision.models.resnet50(pretrained=False)
net = torchvision.models.resnet50(weights=None)

device = "cuda" if torch.cuda.is_available() else "cpu"
net = net.to(device)

### 加载数据集

In [14]:
batch_size = 256
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One ./food11/training sample ./food11/training/0_0.jpg
One ./food11/validation sample ./food11/validation/0_0.jpg


### 开始训练

In [18]:
# The number of training epochs and patience.
n_epochs = 50
patience = 10 # If no improvement in 'patience' epochs, early stop
lr = 0.001
weight_decay = 1e-5

net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

train_start = time.time()
for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    net.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    t0 = time.time()
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = net(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    t1 = time.time()
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    net.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = net(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}, time cost {t1-t0:.3f}s")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a") as f:
            f.write(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a") as f:
            f.write(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(net.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break
        
train_end = time.time()
cost = (train_end-train_start)/60
print(f'Training end, total time cost {cost:.3f}min')

100%|██████████| 39/39 [01:16<00:00,  1.97s/it]


[ Train | 001/050 ] loss = 2.03484, acc = 0.31793


100%|██████████| 14/14 [00:21<00:00,  1.52s/it]


[ Valid | 001/050 ] loss = 2.43437, acc = 0.26709, time cost 76.664s
Best model found at epoch 0, saving model


100%|██████████| 39/39 [01:16<00:00,  1.95s/it]


...

[ Valid | 049/050 ] loss = 1.66371, acc = 0.66915, time cost 78.094s


100%|██████████| 39/39 [01:16<00:00,  1.97s/it]


[ Train | 050/050 ] loss = 0.18149, acc = 0.93636


100%|██████████| 14/14 [00:20<00:00,  1.48s/it]

[ Valid | 050/050 ] loss = 1.56427, acc = 0.67261, time cost 76.872s
Training end, total time cost 82.058min


### 测试与提交CSV

In [19]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One ./food11/test sample ./food11/test/0001.jpg


In [21]:
model_best = torchvision.models.resnet50(weights=None).to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()
        
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)